In [5]:
import pandas as pd
import os

# # Set Working Directory
# os.chdir('K:/Odo/Staff/Connor/NIPPV')

# import dataset
df = pd.read_parquet('output/study_cohort_NIPPV_&_ICU.parquet')

print(f'Total rows loaded: {len(df)}')

admissions = df['hospitalization_id'].nunique()
print(f'Total ICU NIPPV Admissions: {admissions}')

Total rows loaded: 2310545
Total ICU NIPPV Admissions: 6046


In [6]:
df.columns

Index(['hospitalization_id', 'age_at_admission', 'event_time',
       'device_category', 'fio2_set', 'peep_set',
       'peak_inspiratory_pressure_obs', 'tidal_volume_obs', 'pco2_arterial',
       'pco2_venous', 'ph_arterial', 'ph_venous', 'heart_rate', 'height_cm',
       'map', 'respiratory_rate', 'sbp', 'spo2', 'temp_c', 'weight_kg',
       'event_type', 'in_dttm', 'out_dttm', 'location_type',
       'location_category', 'hospital_id', 'hospital_type', 'day_number',
       'hosp_id_day_key', 'gcs_total', 'bmi', 'admission_dttm',
       'discharge_category', 'sex_category'],
      dtype='object')

In [7]:
# Convert to standard pandas datetime
df['event_time'] = pd.to_datetime(df['event_time'])
df['admission_dttm'] = pd.to_datetime(df['admission_dttm'])

# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

In [8]:
vital_sign_cols = ['heart_rate', 'respiratory_rate', 'spo2', 'sbp', 'temp_c']

# Vital signs
vitals = df[df[vital_sign_cols].notna().any(axis=1)]
vitals = vitals.sort_values(['hospitalization_id', 'event_time'])

first_vital = (
    vitals.groupby('hospitalization_id')
          .first()
          .reset_index()[['hospitalization_id', 'event_time']]
          .rename(columns={'event_time': 'first_vital_time'})
)

# NIPPV
nippv = df[df['device_category'] == 'NIPPV']
nippv = nippv.sort_values(['hospitalization_id', 'event_time'])

first_nippv = (
    nippv.groupby('hospitalization_id')
         .first()
         .reset_index()[['hospitalization_id', 'event_time']]
         .rename(columns={'event_time': 'first_nippv_time'})
)

# Merge + compute difference
merged = first_vital.merge(first_nippv, on='hospitalization_id')

merged['time_to_NIPPV'] = (
    merged['first_nippv_time'] - merged['first_vital_time']
).dt.total_seconds() / 3600

# Filter
eligible_ids = merged[merged['time_to_NIPPV'] <= 6]['hospitalization_id']

df = df[df['hospitalization_id'].isin(eligible_ids)]
df = df.merge(
    merged[['hospitalization_id', 'time_to_NIPPV']],
    on='hospitalization_id',
    how='left'
)

print(f'Total rows loaded: {len(df)}')

admissions = df['hospitalization_id'].nunique()
print(f'Total NIPPV < 6 hrs: {admissions}')

Total rows loaded: 455350
Total NIPPV < 6 hrs: 2043


In [9]:
# Calculate nippv_start_time
df['nippv_start_time'] = df['admission_dttm'] + pd.to_timedelta(df['time_to_NIPPV'], unit='h')

# Use nippv_start_time to calculate time_since_nippv
df['time_since_nippv'] = (df['event_time'] - df['nippv_start_time']).dt.total_seconds() / 3600

# Filter to rows where time_since_nippv <= 48 hours
df = df[df['time_since_nippv'] <= 48]

print(f'Total rows loaded: {len(df)}')

admissions = df['hospitalization_id'].nunique()
print(f'Total NIPPV < 6 hrs: {admissions}')

Total rows loaded: 158474
Total NIPPV < 6 hrs: 2042


In [10]:
#Calculate hours_since_admission
df['hours_since_admission'] = (
    df['event_time'] - df['admission_dttm']
    ).dt.total_seconds() / 36000

#Create a dataset with only rows before NIPPV initiaiton
df_PreNIPPV = df[df['hours_since_admission'] < df['time_to_NIPPV']]

# Get the minimum fio2_set for each hospitalization_id prior to NIPPV initiation
min_fio2_PreNIPPV = df_PreNIPPV.groupby('hospitalization_id')['fio2_set'].min().reset_index()

# Identify eligible hospitalizations where min fio2_set <= 60 prior to NIPPV initiation
eligible_fio2 = min_fio2_PreNIPPV[min_fio2_PreNIPPV['fio2_set'] <= 0.60]['hospitalization_id']

# Filter the df to keep only eligible fio2 hospitalizations
df = df[df['hospitalization_id'].isin(eligible_fio2)].reset_index(drop=True)

admissions = df['hospitalization_id'].nunique()
print(f'Total NIPPV < 6 hrs & fio2_set <= 0.60: {admissions}')

Total NIPPV < 6 hrs & fio2_set <= 0.60: 1626


In [11]:
#Update df_PreNIPPV with only filtered admissions
df_PreNIPPV = df[df['hours_since_admission'] < df['time_to_NIPPV']]

# Get maximum pco2_arterial and pco2_venous prior to NIPPV initiation for each hospitalization_id
max_pco2_arterial = df_PreNIPPV.groupby('hospitalization_id')['pco2_arterial'].max()
max_pco2_venous = df_PreNIPPV.groupby('hospitalization_id')['pco2_venous'].max()

# Combine max_pco2_arterial and max_pco2_venous into one dataframe
max_pco2 = pd.DataFrame({
    'pco2_arterial': max_pco2_arterial,
    'pco2_venous': max_pco2_venous
}).reset_index()

# Filter rows where max_pco2_arterial or max_pco2_venous >= 45
eligible_pco2 = max_pco2[
    (max_pco2['pco2_arterial'] >= 45) | (max_pco2['pco2_venous'] >= 45)
]['hospitalization_id']

print(f'Total NIPPV < 6 hrs & fio2_set <= 60 & pco2 >= 45: {len(eligible_pco2)}')

# Filter the df to keep only eligible pco2 hospitalizations
df = df[df['hospitalization_id'].isin(eligible_pco2)].reset_index(drop=True)

Total NIPPV < 6 hrs & fio2_set <= 60 & pco2 >= 45: 1030


In [12]:
#Update df_PreNIPPV with only filtered admissions
df_PreNIPPV = df[df['hours_since_admission'] < df['time_to_NIPPV']]

# Get minimum ph_arterial and ph_venous prior to NIPPV initiation for each hospitalization_id
min_ph_arterial = df_PreNIPPV.groupby('hospitalization_id')['ph_arterial'].min()
min_ph_venous = df_PreNIPPV.groupby('hospitalization_id')['ph_venous'].min()

# Combine min_ph_arterial and min_ph_venous into one dataframe
min_ph = pd.DataFrame({
    'ph_arterial': min_ph_arterial,
    'ph_venous': min_ph_venous
}).reset_index()

# Filter to rows where min_ph_arterial and min_ph_venous <= 7.35
eligible_ph = min_ph[
    (min_ph['ph_arterial'] <= 7.35) | (min_ph['ph_venous'] <= 7.35)
]['hospitalization_id']

print(f'Total NIPPV < 6 hrs & fio2_set <= 60 & pco2 >= 45 & ph <= 7.35: {len(eligible_ph)}')

# Filter the df to keep only eligible ph hospitalizations
df = df[df['hospitalization_id'].isin(eligible_ph)].reset_index(drop=True)

Total NIPPV < 6 hrs & fio2_set <= 60 & pco2 >= 45 & ph <= 7.35: 755


In [13]:
# Create a new dataframe of IMV events
imv_df = df[df['device_category'] == 'IMV'].copy()
# Identify IMV hospitalizations
imv_ids = imv_df['hospitalization_id'].unique()

# Create a new dataframe of patients who died
expired_df = df[df['discharge_category'] == 'Expired'].copy()
# Identify dead patients
expired_ids = expired_df['hospitalization_id'].unique()

# Create separate failure flags
df['failure_imv'] = df['hospitalization_id'].isin(imv_ids).astype(int)
df['failure_death'] = df['hospitalization_id'].isin(expired_ids).astype(int)

# Overall failure: either IMV or death
df['failure'] = ((df['failure_imv'] == 1) | (df['failure_death'] == 1)).astype(int)

# Count totals (unique hospitalizations)
total_patients = df['hospitalization_id'].nunique()
total_failures = df[df['failure'] == 1]['hospitalization_id'].nunique()
imv_failures = df[df['failure_imv'] == 1]['hospitalization_id'].nunique()
death_failures = df[df['failure_death'] == 1]['hospitalization_id'].nunique()

print(f'Total patients: {total_patients}')
print(f'Total failures: {total_failures}')
print(f'IMV failures: {imv_failures}')
print(f'Deaths: {death_failures}')

Total patients: 755
Total failures: 160
IMV failures: 133
Deaths: 52


In [14]:
#Create a dataset with only rows after NIPPV initiaiton
df_PostNIPPV = df[df['event_time'] >= df['nippv_start_time']]

# Sort df_PostNIPPV to ensure earliest events come first
df_PostNIPPV = df_PostNIPPV.sort_values(['hospitalization_id', 'event_time'])

# Get the first heart_rate after NIPPV for each hospitalization
heart_rate = df_PostNIPPV.groupby('hospitalization_id')['heart_rate'].first().reset_index()
heart_rate = heart_rate.rename(columns={'heart_rate': 'heart_rate_after_NIPPV'})

df = df.merge(heart_rate, on='hospitalization_id', how='left')

In [15]:
# Resort df_PostNIPPV to ensure earliest events come first
df_PostNIPPV = df_PostNIPPV.sort_values(['hospitalization_id', 'event_time'])

# Get the first heart_rate after NIPPV for each hospitalization
respiratory_rate = df_PostNIPPV.groupby('hospitalization_id')['respiratory_rate'].first().reset_index()
respiratory_rate = respiratory_rate.rename(columns={'respiratory_rate': 'respiratory_rate_after_NIPPV'})

df = df.merge(respiratory_rate, on='hospitalization_id', how='left')

In [16]:
# Resort df_PostNIPPV to ensure earliest events come first
df_PostNIPPV = df_PostNIPPV.sort_values(['hospitalization_id', 'event_time'])

# Create combined PostNIPPV_pco2_combined column
df_PostNIPPV['PostNIPPV_pco2_combined'] = df_PostNIPPV['pco2_arterial'].combine_first(df_PostNIPPV['pco2_venous'])

# Sort by hospitalization and event time
df = df.sort_values(['hospitalization_id', 'event_time'])

# Get first pco2_combined after NIPPV for each hospitalization
pco2_PostNIPPV = df_PostNIPPV.groupby('hospitalization_id')['PostNIPPV_pco2_combined'].first().reset_index()
pco2_PostNIPPV = pco2_PostNIPPV.rename(columns={'PostNIPPV_pco2_combined': 'pco2_after_NIPPV'})

# Merge back into main df
df = df.merge(pco2_PostNIPPV, on='hospitalization_id', how='left')

In [17]:
# Resort df_PostNIPPV to ensure earliest events come first
df_PostNIPPV = df_PostNIPPV.sort_values(['hospitalization_id', 'event_time'])

# Get the first fio2 after NIPPV for each hospitalization
fio2_PostNIPPV = df_PostNIPPV.groupby('hospitalization_id')['fio2_set'].first().reset_index()
fio2_PostNIPPV = fio2_PostNIPPV.rename(columns={'fio2_set': 'fio2_after_NIPPV'})

df = df.merge(fio2_PostNIPPV, on='hospitalization_id', how='left')

In [18]:
# Create combined pco2 column
df['pco2_combined'] = df['pco2_arterial'].combine_first(df['pco2_venous'])

# Sort by hospitalization and event time
df = df.sort_values(['hospitalization_id', 'event_time'])

# Get initial pco2_combined for each hospitalization
initial_pco2 = df.groupby('hospitalization_id')['pco2_combined'].first().reset_index()
initial_pco2 = initial_pco2.rename(columns={'pco2_combined': 'first_pco2'})

# Merge back into main df
df = df.merge(initial_pco2, on='hospitalization_id', how='left')

In [19]:
# Create combined ph column
df['ph_combined'] = df['ph_arterial'].combine_first(df['ph_venous'])

# Sort by hospitalization and event time
df = df.sort_values(['hospitalization_id', 'event_time'])

# Get initial ph_combined for each hospitalization
initial_ph = df.groupby('hospitalization_id')['ph_combined'].first().reset_index()
initial_ph = initial_ph.rename(columns={'ph_combined': 'first_ph'})

# Merge back into main df
df = df.merge(initial_ph, on='hospitalization_id', how='left')

In [20]:
df_analytic = df.groupby('hospitalization_id').agg({
    'age_at_admission': 'first',
    'sex_category': 'first',
    'bmi': 'max',
    'map': 'median',
    'first_pco2': 'first',
    'first_ph': 'first',
    'peep_set': 'first',
    'peak_inspiratory_pressure_obs': 'first',
    'tidal_volume_obs': 'first',
    'heart_rate_after_NIPPV': 'first',
    'respiratory_rate_after_NIPPV': 'first',
    'pco2_after_NIPPV': 'first',
    'fio2_after_NIPPV': 'first',
    'failure_imv':'first',
    'failure_death':'first',
    'failure': 'first'
}).reset_index()


In [21]:
df.columns

Index(['hospitalization_id', 'age_at_admission', 'event_time',
       'device_category', 'fio2_set', 'peep_set',
       'peak_inspiratory_pressure_obs', 'tidal_volume_obs', 'pco2_arterial',
       'pco2_venous', 'ph_arterial', 'ph_venous', 'heart_rate', 'height_cm',
       'map', 'respiratory_rate', 'sbp', 'spo2', 'temp_c', 'weight_kg',
       'event_type', 'in_dttm', 'out_dttm', 'location_type',
       'location_category', 'hospital_id', 'hospital_type', 'day_number',
       'hosp_id_day_key', 'gcs_total', 'bmi', 'admission_dttm',
       'discharge_category', 'sex_category', 'time_to_NIPPV',
       'nippv_start_time', 'time_since_nippv', 'hours_since_admission',
       'failure_imv', 'failure_death', 'failure', 'heart_rate_after_NIPPV',
       'respiratory_rate_after_NIPPV', 'pco2_after_NIPPV', 'fio2_after_NIPPV',
       'pco2_combined', 'first_pco2', 'ph_combined', 'first_ph'],
      dtype='object')

In [22]:
#Drop missing values
df_analytic_clean = df_analytic

In [23]:
admissions = df_analytic['hospitalization_id'].nunique()
print(f'Cohort hospitalizations with misising data: {admissions}')

admissions = df_analytic_clean['hospitalization_id'].nunique()
print(f'Cohort hospitalizations without misising data: {admissions}')

Cohort hospitalizations with misising data: 755
Cohort hospitalizations without misising data: 755


In [24]:
# Export final df to CSV
df_analytic_clean.to_csv('output/NIPPV_analytic_dataset.csv', index=False)

In [25]:
df_analytic_clean.columns

Index(['hospitalization_id', 'age_at_admission', 'sex_category', 'bmi', 'map',
       'first_pco2', 'first_ph', 'peep_set', 'peak_inspiratory_pressure_obs',
       'tidal_volume_obs', 'heart_rate_after_NIPPV',
       'respiratory_rate_after_NIPPV', 'pco2_after_NIPPV', 'fio2_after_NIPPV',
       'failure_imv', 'failure_death', 'failure'],
      dtype='object')

In [26]:
df_analytic_clean.isna().sum()

hospitalization_id                0
age_at_admission                  0
sex_category                      0
bmi                              24
map                               0
first_pco2                        0
first_ph                          0
peep_set                          5
peak_inspiratory_pressure_obs    14
tidal_volume_obs                  3
heart_rate_after_NIPPV            0
respiratory_rate_after_NIPPV      0
pco2_after_NIPPV                 47
fio2_after_NIPPV                  9
failure_imv                       0
failure_death                     0
failure                           0
dtype: int64

In [27]:
# Total number of rows
total_rows = len(df_analytic_clean)

# Total number of failures (failure == 1)
total_failures = df_analytic_clean['failure'].sum()
imv_failures = df_analytic_clean['failure_imv'].sum()
death_failures = df_analytic_clean['failure_death'].sum()
both_failures = df_analytic_clean[(df_analytic_clean['failure_imv'] == 1) & (df_analytic_clean['failure_death'] == 1)]['hospitalization_id'].nunique()

print(f'Total rows: {total_rows}')
print(f'Total failures: {total_failures}')
print(f'Total IMV failures: {imv_failures - both_failures}')
print(f'Total death failures: {death_failures - both_failures}')
print(f'Both failures: {both_failures}')

Total rows: 755
Total failures: 160
Total IMV failures: 108
Total death failures: 27
Both failures: 25


In [ ]:

# ######################MIMIC
# Total rows: 259
# Total failures: 101
# Total IMV failures: 65
# Total death failures: 24
# Both failures: 12


# hospitalization_id                 0
# age_at_admission                   0
# sex_category                       0
# bmi                              140
# map                                0
# first_pco2                         0
# first_ph                           0
# peep_set                           5
# peak_inspiratory_pressure_obs     64
# tidal_volume_obs                  50
# heart_rate_after_NIPPV             0
# respiratory_rate_after_NIPPV       0
# pco2_after_NIPPV                   2
# fio2_after_NIPPV                   3
# failure_imv                        0
# failure_death                      0
# failure                            0


# ######################RUSH
# hospitalization_id                0
# age_at_admission                  0
# sex_category                      0
# bmi                              24
# map                               0
# first_pco2                        0
# first_ph                          0
# peep_set                          5
# peak_inspiratory_pressure_obs    14
# tidal_volume_obs                  3
# heart_rate_after_NIPPV            0
# respiratory_rate_after_NIPPV      0
# pco2_after_NIPPV                 47
# fio2_after_NIPPV                  9
# failure_imv                       0
# failure_death                     0
# failure                           0

# Total rows: 755
# Total failures: 160
# Total IMV failures: 108
# Total death failures: 27
# Both failures: 25

SyntaxError: invalid syntax (1152774133.py, line 1)